# Factorization Machines

Factorization machines can be compared to support vector machines (SVMs) with a polynomial kernel, according to Rendle and others. This algorithm has been well-studied and evaluated. The interesting thing behind support vector machines is that they are still somewhat of a mystery. They are often verified empirically rather than against a theoretical baseline or limit. Even so, they are widely used as general predictors. SVMs define a multidimensional hyperplane, which learns the shape of the curve of the data.

However, SVMs have known weaknesses, some of which are addressed by Rendle’s factorization machines. For example, they require that the output model contain support vectors, which are training examples that help the algorithm define the shape of the hyperplane as well as other parameters such as the margin for prediction.

**SVMs function best on dense data** — that is, data with few to no missing values and data whose plotted points fall near each other. Finally, in SVMs, the input variables are still independent variables even though the polynomial kernel attempts to model the interaction among the variables. This is computed in polynomial time based upon the number of dimensions in the input data as well as the order of the polynomial (e.g., quadratic, cubic or quartic).

Factorization machines were designed to address these weaknesses. Firstly, no training examples are required in the model parameters, making the models much more compact. Factorization machines perform extremely well on sparse data, including data of very high sparsity. As a trade-off, they do not perform well on dense data, so other algorithms are more suited to this class of data.

Finally, and perhaps most amazing of all, factorization machines can model n-way variable interactions, where n is the number of polynomial order. Note that in all current implementations, the order has been fixed at two. While the equations generalize to higher orders, there are some issues such as the numerical stability of the optimization methods that have prevented the generalization of n.

Rendle ingeniously proposed a method of reducing the polynomial computation time to linear complexity. It is this capability of factorization machines that makes them extremely attractive to researchers.

Additionally, **Rendle demonstrates that, through proper feature engineering, the machines can mimic the best specialized factorization models developed for very specific situations. This allows them to be applied in a multitude of situations where a specific form of learning algorithm and predictor are required.**

In his original paper, Rendle discussed a method of optimization for the model parameters known as **stochastic gradient descent, which works well with several loss functions. However, the optimization algorithm is extremely dependent on the learning rate, one of the hyperparameters of the method. If the learning rate is too high, the model parameters will not converge, while if it is too low, the algorithm is no longer time-efficient.**

**Because of this, Rendle reviewed three more methods of optimization in “Factorization Machines With LibFM,” known as alternating least-squares, marcov chain monte carlo inference and adaptive stochastic gradient descent. He recommends marcov chain monte carlo inference because there are fewer hyperparameters, and those that must be specified are not as sensitive to their initial values.**

**Factorization machines have been widely applied in the field of collaborative recommendation systems, where their sparse predictive power and ability to mimic state-of-the-art, specific factorization methods make them generalizable to several tasks.** Not only are they used in recommending movies and music, for example, but researchers have even applied them to use social media to predict the stock market with surprisingly positive results.

As I discussed above, factorization machines can function as **general predictors akin to support vector machines in high-dimensional sparse data.** While they are not commonly used outside of recommendation systems at present, they have the potential to become classifiers similar to SVMs.

There are several implementations of factorization machines. **The state-of-the-art continues to be libFM (http://www.libfm.org/),** created by Rendle himself. However, there are other interesting implementations such as **fastFM (https://github.com/ibayer/fastFM).** There’s even a version of libFM designed for the Spark framework, known as **spark-libFM (https://github.com/zhengruifeng/spark-libFM),** that allows factorization machines to be parallelized.

## FastFM

This repository allows you to use Factorization Machines in Python (2.7 & 3.x) with the well known scikit-learn API. All performence critical code as been written in C and wrapped with Cython. fastFM provides **stochastic gradient descent (SGD) and coordinate descent (CD) optimization routines as well as Markov Chain Monte Carlo (MCMC)** for Bayesian inference. 

In [13]:
import numpy as np
import pandas as pd
from fastFM import als
from scipy import sparse

<pre>
```

|     Task       |      Solver     |            Loss              |
|    ------      |      ------     |           ------             |
|  Regression    | als, mcmc, sgd  |        Square Loss           |
|  Classification| als, mcmc, sgd  | Probit(Map), Probit, Sigmoid |
|  Ranking       | sgd             |             BPR              |

```
<pre>

## Regression with ALS Solver

In [7]:
df = pd.read_csv("/home/antonio/Dropbox/DataAnalysis/python/dati/glass.data.csv")

In [8]:
df.head()

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [9]:
df.drop("Id number",axis=1,inplace=True)

In [11]:
X_train = df.drop("Type of glass",axis=1)
y_train = df["Type of glass"]

In [26]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(sparse.bsr_matrix(X_train), y_train)
y_pred = fm.predict(sparse.bsr_matrix(X_train))

## Logit Classification with SGD Solver

We first have to convert the target of our toy dataset to -1/1 values in order to work with the classification implementation. Currently only binary classification is supported.

In [32]:
from fastFM.datasets import make_user_item_regression
from sklearn.cross_validation import train_test_split

# This sets up a small test dataset.
X, y, _ = make_user_item_regression(label_stdev=.4)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [33]:
# Convert dataset to binary classification task.
y_labels = np.ones_like(y)
y_labels[y < np.mean(y)] = -1
X_train, X_test, y_train, y_test = train_test_split(X, y_labels)

We could have used the ALS solver module for this problem as well but we will use the SGD module instead. In addition to the hyper parameter needed for the ALS module we need to specify the SGD specific step_size parameter.

In [40]:
from fastFM import sgd
fm = sgd.FMClassification(n_iter=1000, init_stdev=0.1, l2_reg_w=0,
                          l2_reg_V=0, rank=2, step_size=0.1)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)
y_pred_proba = fm.predict_proba(X_test)

In [41]:
from sklearn.metrics import accuracy_score, roc_auc_score
'acc:', accuracy_score(y_test, y_pred)
'auc:', roc_auc_score(y_test, y_pred_proba)

('auc:', 0.99278846153846156)

## Bayesian Probit Classification with MCMC Solver

The MCMC module needs fewer hyper parameter that any other solver. This solver is able to integrate out the regularization parameter and frees us from selecting them manually. Please see [Freuden2011] for the detail on the implemented Gibbs sampler. The major drawback of the MCMC solver is that it forces us to calculate predictions during fitting time using the fit_predict function. It’s however possible to select a subset of parameter draws to speed up prediction [RecSys2013]. It’s also possible to just call predict on a trained MCMC model but this returns predictions that are solely based on the last parameters draw. These predictions can be used for diagnostic purposes but are usually not as good as averaged predictions returned by fit_predict.

In [42]:
from fastFM import mcmc
fm = mcmc.FMClassification(n_iter=1000, rank=2, init_stdev=0.1)

Let's see how to use the MCMC module for binary classification. Probit regression uses the Cumulative Distribution Function (CDF) of the standard normal Distribution as link function. Mainly because the CDF leads to an easier Gibbs solver then the sigmoid function used in the SGD classifier implementation. The results are in practice usually very similar.

In [43]:
y_pred = fm.fit_predict(X_train, y_train, X_test)
y_pred_proba = fm.fit_predict_proba(X_train, y_train, X_test)

In [44]:
'acc:', accuracy_score(y_test, y_pred)
'auc:', roc_auc_score(y_test, y_pred_proba)

('auc:', 0.99439102564102566)